In [51]:
import data
import pandas as pd
import math
import plotly.graph_objects as go
import numpy as np
import plotly
import pprint
#from tools import write_pandas_to_gsheet

MERGE_NO_US, MERGED_CSBS_JHU, JHU_TIME, JHU_RECENT, DATE_MAPPER, CSBS, CENTROID_MAPPER = data.get_data() 

In [43]:
JHU_DF_AGG_COUNTRY = JHU_TIME.sort_values('confirmed')[::-1].groupby(['Date','country']).agg(
            {'lat': 'first', 'lon': 'first', 'confirmed': 'sum', 'deaths': 'sum'}).reset_index()

JHU_DF_AGG_PROVINCE = JHU_TIME[JHU_TIME['province']!=''].sort_values('confirmed')[::-1].groupby(['Date','province']).agg(
            {'lat': 'first', 'lon': 'first', 'confirmed': 'sum', 'deaths': 'sum'}).reset_index()

CSBS_DF_AGG_STATE = CSBS[CSBS['province'] != ''].sort_values('confirmed')[::-1].groupby(['Date','province']).agg(
            {'lat': 'first', 'lon': 'first', 'confirmed': 'sum', 'deaths': 'sum'}).reset_index().rename({'province': 'state'}, axis=1)

CSBS_DF_AGG_COUNTY = CSBS[CSBS['county'] != ''].sort_values('confirmed')[::-1].groupby(['Date','county']).agg(
            {'lat': 'first', 'lon': 'first', 'confirmed': 'sum', 'deaths': 'sum'}).reset_index()

In [92]:
how_many_labels = (len(DATE_MAPPER))//10
marks = {k: {'label': v}
         for k, v in list(DATE_MAPPER['Date'].dt.strftime(
             '%m/%d/%y').to_dict().items())[::how_many_labels]}
last_key = list(marks.keys())[-1]
todays_key = DATE_MAPPER.index[-1]

In [93]:
marks

{0: {'label': '01/22/20'},
 6: {'label': '01/28/20'},
 12: {'label': '02/03/20'},
 18: {'label': '02/09/20'},
 24: {'label': '02/15/20'},
 30: {'label': '02/21/20'},
 36: {'label': '02/27/20'},
 42: {'label': '03/04/20'},
 48: {'label': '03/10/20'},
 54: {'label': '03/16/20'},
 60: {'label': '03/22/20'},
 66: {'label': '03/28/20'}}

In [91]:
def update_total_case_graph(values):
    colors = plotly.colors.qualitative.Dark24
    pprint.pprint(values)
    fig = go.Figure()
    for enum_, item in enumerate(values):
        color_ = colors[enum_]
        if item == 'worldwide':
            sub_df = JHU_DF_AGG_COUNTRY.groupby('Date').sum().reset_index()
            name = 'World'
        else:
            if item.split('_')[0] == 'COUNTRY':
                name = item.split('_')[1]
                sub_df = JHU_DF_AGG_COUNTRY[JHU_DF_AGG_COUNTRY['country'] == name].groupby(
                    'Date').sum().reset_index()

            elif item.split('_')[0] == 'PROVINCE':
                name = item.split('_')[1]
                sub_df = JHU_DF_AGG_PROVINCE[JHU_DF_AGG_PROVINCE['province'] == name].groupby(
                    'Date').sum().reset_index()

            elif item.split('_')[0] == 'STATE':
                name = item.split('_')[1]
                sub_df = CSBS_DF_AGG_STATE[CSBS_DF_AGG_STATE['state'] == name].groupby(
                    'Date').sum().reset_index()

            elif item.split('_')[0] == 'COUNTY':
                name = item.split('_')[1]
                sub_df = CSBS_DF_AGG_COUNTY[CSBS_DF_AGG_COUNTY['county'] == name].groupby(
                    'Date').sum().reset_index()
            else:
                raise Exception('You have messed up {}'.format(item))

        fig.add_trace(
            go.Scatter(
                x=sub_df['Date'],
                y=sub_df['confirmed'],
                name=name,
                showlegend=True,
                mode='lines+markers',
                hovertemplate='%{x}<br>Confirmed Cases - %{y:,f}',
                marker=dict(
                    color=color_,
                    size=2,
                    line=dict(
                        width=5,
                        color=color_),
                )))

    fig.update_layout(
        # xaxis_tickfont_size=22,
        yaxis=dict(
            title=dict(text='Total Cases', standoff=2),
            titlefont_size=12,
            tickfont_size=12,
            showgrid=True,
            color='white',
            side='left',
        ),
        xaxis=dict(
            color='white'
        ),
        autosize=True,
        showlegend=True,
        legend=dict(x=0,y=1,font=dict(color='white')),
        paper_bgcolor='#1f2630',
        plot_bgcolor='rgb(52,51,50)',
        margin=dict(t=20,pad=10,b=20,r=20,l=20))

    return fig







def per_day_case_graph(values):
    colors = plotly.colors.qualitative.Dark24
    pprint.pprint(values)
    fig = go.Figure()
    #fig = plotly.subplots.make_subplots(specs=[[{"secondary_y": True}]])
    for enum_, item in enumerate(values):
        color_ = colors[enum_]
        if item == 'worldwide':
            sub_df = JHU_DF_AGG_COUNTRY.groupby('Date').sum().reset_index()
            name = 'World'
        else:
            if item.split('_')[0] == 'COUNTRY':
                name = item.split('_')[1]
                sub_df = JHU_DF_AGG_COUNTRY[JHU_DF_AGG_COUNTRY['country'] == name].groupby(
                    'Date').sum().reset_index()

            elif item.split('_')[0] == 'PROVINCE':
                name = item.split('_')[1]
                sub_df = JHU_DF_AGG_PROVINCE[JHU_DF_AGG_PROVINCE['province'] == name].groupby(
                    'Date').sum().reset_index()

            elif item.split('_')[0] == 'STATE':
                name = item.split('_')[1]
                sub_df = CSBS_DF_AGG_STATE[CSBS_DF_AGG_STATE['state'] == name].groupby(
                    'Date').sum().reset_index()

            elif item.split('_')[0] == 'COUNTY':
                name = item.split('_')[1]
                sub_df = CSBS_DF_AGG_COUNTY[CSBS_DF_AGG_COUNTY['county'] == name].groupby(
                    'Date').sum().reset_index()
            else:
                raise Exception('You have messed up {}'.format(item))

        xs = sub_df['Date']
        ys = sub_df['confirmed'].diff().fillna(0)
        fig.add_trace(
            go.Bar(
                x=xs,
                y=ys,
                name=name,
                showlegend=True,
                #text=ys,
                #textposition='outside',
                hovertemplate='%{x}<br>New Cases - %{y:,f}',
                #texttemplate='%{text:.2s}',
                textfont=dict(size=14,color='white'),
                marker=dict(
                    color=color_,
                    line=dict(
                        color='white',width=0.5)
                )))


    fig.update_layout(
        yaxis=dict(
            title=dict(text='New Cases', standoff=2),
            titlefont_size=12,
            tickfont_size=12,
            showgrid=True,
            color='white',
            side='left',
        ),
        xaxis=dict(
            color='white'
        ),
        showlegend=True,
        legend=dict(x=0,y=1,font=dict(color='white')),
        paper_bgcolor='#1f2630',
        plot_bgcolor='rgb(52,51,50)',
        barmode='group',
        bargap=0.1,
        margin=dict(t=20,pad=10,b=20,r=20,l=20))  




    return fig


# 

update_total_case_graph(['worldwide','COUNTRY_US','COUNTRY_Italy'])

['worldwide', 'COUNTRY_US', 'COUNTRY_Italy']
